In [1]:
import cPickle as pickle
# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup

In [2]:
import nltk
from nltk.stem.porter import *

In [3]:
import numpy as np

In [20]:
import TFIDF_numpy as tfidf

In [42]:
tfidf = reload(tfidf)

In [43]:
dir(tfidf)

['__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 'asdf',
 'calculate_cossim',
 'calculate_tfidf',
 'calculate_tfidfs',
 'create_tfs',
 'get_tfs',
 'idf_vector',
 'load_indices',
 'load_questions',
 'np',
 'question_texts',
 'tf_vectors',
 'tfidf_norms',
 'tfidf_vectors',
 'word_indices']

In [5]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [6]:
questions = pickle.load(open("lstOfQuestions.pkl", "rb"))

In [7]:
questions = questions[:20000]

In [8]:
words_dict = {}
question_texts = []
total = len(questions)
count = 0
for question in questions :
    if count % (total / 10) == 0:
        print count
    VALID_TAGS = ['p']

    soup = BeautifulSoup(question['body'])

    VALID_TAGS = ['p']
    INVALID_TAGS = ['code', 'a']

    text = ' '.join([s.get_text() for s in soup('p')])

    tokens = nltk.word_tokenize(text)

    stemmer = PorterStemmer()

    words = [stemmer.stem(token.lower()) for token in tokens]

    question_texts.append(words)
    
    for word in words :
        if word in words_dict :
            words_dict[word] += 1
        else :
            words_dict[word] = 1
    
    count += 1

0
2000
4000
6000
8000
10000
12000
14000
16000
18000


In [9]:
id_hash = {}
count = 0
for question in questions:
    if question['question_id'] not in id_hash:
        id_hash[question['question_id']] = 1
        count += 1

print count

20000


In [10]:
# Go through dictionary, removing entries that have a value less
# than delta
delta = 2
words_dict2 = {key: value for key, value in words_dict.items()
                 if value >= delta }

word_indices = {}
index = 0
for word in words_dict2 :
    word_indices[word] = index
    index += 1

In [12]:
output = open('wordIndices.pkl', 'wb')
pickle.dump(word_indices, output, -1)
output.close()

In [33]:
# Determine Term Frequencies
tf_vectors = []
for text in question_texts :
    vector = np.zeros(len(word_indices))
    for word in text :
        if word in word_indices :
            vector[word_indices[word]] += 1
    tf_vectors.append(vector)
tf_vectors = np.array(tf_vectors)
tfidf.get_tfs()

In [34]:
# Determine IDFS
num_docs_vector = np.zeros(len(word_indices))
N = float(len(questions))
for text in question_texts :
    temp_dict = {}
    for word in text :
        if word in word_indices :
            if word in temp_dict :
                continue
            else :
                temp_dict[word] = True
                num_docs_vector[word_indices[word]] += 1

idf_vector = np.log(N / num_docs_vector)                
tfidf_vectors = tf_vectors * idf_vector[None, :]
tfidf_norms = np.linalg.norm(tfidf_vectors, axis=1)

In [35]:
example = tfidf_vectors[0]

In [37]:
# results = tfidf_vectors.dot(example) / (np.linalg.norm(example) * np.ones(20000))
tfidf_vectors.dot(example) / (np.linalg.norm(example) * tfidf_norms)

/Users/georgelok/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in divide
  from ipykernel import kernelapp as app


array([ 1.        ,  0.0158244 ,  0.04565059, ...,  0.01414114,
        0.02306023,  0.11155965])

In [44]:
tfidf.load_questions(question_texts)
tfidf.load_indices(word_indices)
tfidf.create_tfs()
tfidf.calculate_tfidfs()
tfidf.calculate_cossim(question_texts[0])

TFIDF_numpy.py:75: RuntimeWarning: invalid value encountered in divide


array([ 1.        ,  0.0158244 ,  0.04565059, ...,  0.01414114,
        0.02306023,  0.11155965])

In [ ]:
import xxhash


In [ ]:
simhashes = []
for u in range(len(question_texts)):
    W = np.zeros(64)
    for i in range(len(question_texts[u])):
#         print (question_texts[u][i])
        try:
            wordhash = xxhash.xxh64(question_texts[u][i]).intdigest()
            counter = 63
            while wordhash > 0:
                bit = wordhash % 2
                if bit:
                    W[counter] += tfidf_vectors[u][i]
                else:
                    W[counter] -= tfidf_vectors[u][i]
                wordhash = wordhash >> 1
                counter -= 1
        except:
            pass
    simhash = 0
    for i in range(len(W)):
        if W[i] >= 0:
            simhash += 1
        if i < len(W)-1:
            simhash = simhash << 1
    simhashes.append(simhash)

In [ ]:
simhashes[10]




In [ ]:
for i in xrange(len(simhashes)):
    for j in xrange(i, len(simhashes)):
        if i != j:
            